In [ ]:
# 假设 experiments_results 是你的三层嵌套列表，这里用一个示例列表表示
experiments_results = [
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]],
    [[13, 14, 15], [16, 17, 18]],
    [[19, 20, 21], [22, 23, 24]],
    [[25, 26, 27], [28, 29, 30]],
    [[31, 32, 33], [34, 35, 36]],
    [[37, 38, 39], [40, 41, 42]],
    [[43, 44, 45], [46, 47, 48]],
    [[49, 50, 51], [52, 53, 54]],
    [[55, 56, 57], [58, 59, 60]]
]

# 使用 zip(*experiments_results) 获取每个实验对应位置的结果，然后对每个位置的结果求平均值
averaged_results = [
    [sum(values) / len(values) for values in zip(*experiment)]
    for experiment in zip(*experiments_results)
]

print(averaged_results)

In [ ]:
list_2 = [[1,2],[4,5]]
len(list_2)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 这里保证了self.conv1和self.bn1的参数在设备上
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False).to(device)
        self.bn1 = nn.BatchNorm2d(64).to(device)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512*block.expansion, num_classes).to(device)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layer = block(self.in_planes, planes, stride)
            layer = layer.to(device)  # 将子模块移动到设备上
            layers.append(layer)
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out1 = out
        out = self.layer2(out)
        out2 = out
        out = self.layer3(out)
        out3 = out
        out = self.layer4(out)
        out4 = out
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return x, out1, out2, out3, out4, out
    
    def children(self):
        # Return an iterator over child modules
        return iter([self.layer1, self.layer2, self.layer3, self.layer4, self.avgpool])

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

# 对模型参数进行高斯分布的随机初始化
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        init.normal_(m.weight.data, mean=0, std=0.01)
        if m.bias is not None:
            init.constant_(m.bias.data, 0)  # 如果有偏置，也进行初始化

# 使用示例
net = ResNet18()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(net)
# 对模型的所有权重参数应用初始化
net.to(device)
net.apply(init_weights)


# 示例输入
sample_input = torch.randn(64, 3, 32, 32).to(device)  # 输入大小为(批量大小, 通道数, 高度, 宽度)

# 前向传播
original_input, hidden_output1, hidden_output2, hidden_output3, hidden_output4, output = net(sample_input)
# print("Output shape:", output.shape)
# print("Original input shape:", original_input.shape)
# print("Hidden output shape:", hidden_output1.shape)

# 打印模型参数
for name, param in net.named_parameters():
    # print(name, param.data)
    print(f"Parameter '{name}' device: {param.device}")

In [ ]:
import torch
import torchvision.models as models

# 加载预训练的ResNet模型
model = models.resnet50(pretrained=False)
# model.apply(init_weights)

# 获取模型的层级信息
print(model)

# 获取中间隐藏层的输出
# 假设我们想要获取第4个卷积层的输出
desired_layer = model.layer3[4]

# 创建一个新的PyTorch模型，使其输出你所需的层的特征
new_model = torch.nn.Sequential(
    *list(model.children())[:8],  # 选择ResNet的前8层（包括你想要的隐藏层）
    desired_layer
)

# 将模型设置为评估模式
new_model.eval()

# 假设有输入数据x
x = torch.randn(1, 3, 224, 224)

# 通过新模型获取隐藏层的输出
hidden_output = new_model(x)
print(hidden_output.shape)  # 输出隐藏层的特征张量的形状


In [3]:
import pickle
from pprint import pprint
# 读取.pkl文件
file_path = r'.\angle_layer_out\2\LeNet\5\betti_number.pkl'  # 替换成你的.pkl文件路径
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# 打印读取的内容
pprint(data)


{'BD-L1': [array([[0.        , 0.16313615],
       [0.        , 0.17169461],
       [0.        , 0.17249253],
       ...,
       [0.        , 0.7652908 ],
       [0.        , 0.76897693],
       [0.        ,        inf]]),
           array([[0.77976888, 0.79159474],
       [0.77660131, 0.83017969],
       [0.75985825, 0.863967  ],
       ...,
       [0.28851199, 0.35552841],
       [0.28678674, 0.29192063],
       [0.22622396, 0.24092001]])],
 'BD-L2': [array([[0.00048828, 0.06435132],
       [0.00048828, 0.06774715],
       [0.        , 0.06875626],
       ...,
       [0.00034527, 0.27529818],
       [0.        , 0.28107944],
       [0.        ,        inf]]),
           array([[0.31022215, 0.31190911],
       [0.29408622, 0.31028497],
       [0.29161584, 0.29472134],
       ...,
       [0.10988851, 0.13080561],
       [0.10924359, 0.11792169],
       [0.10287797, 0.10297006]])]}


In [4]:

file_path2 = r'.\angle_layer_out\10\ResNet152\compare_different_bitte_norm_in_same_augmentation.pkl'  # 替换成你的.pkl文件路径
with open(file_path2, 'rb') as file2:
    data2 = pickle.load(file2)

# 打印读取的内容
pprint(data2)

OrderedDict([('L1-B0',
              {'all_bars_survive_time_sum': {'ResNet152\\0': 33796.9842376709,
                                             'ResNet152\\1': 79204.99718475342,
                                             'ResNet152\\2': 47895.405391693115,
                                             'ResNet152\\3': 23559.97602081299,
                                             'ResNet152\\4': 13276.59432220459,
                                             'ResNet152\\5': 2.4287756928242743},
               'bar_number': {'ResNet152\\0': 960,
                              'ResNet152\\1': 960,
                              'ResNet152\\2': 960,
                              'ResNet152\\3': 960,
                              'ResNet152\\4': 960,
                              'ResNet152\\5': 960},
               'birth_len': {'ResNet152\\0': 0.0,
                             'ResNet152\\1': 0.0,
                             'ResNet152\\2': 0.0,
                             'ResNet15

In [4]:
import plotly.graph_objs as go
import plotly.io as pio
import numpy as np
x = random_array_1d = np.random.rand(100)  # 生成100个0到1之间的随机数
y = random_array_1d = np.random.rand(100)  # 生成100个0到1之间的随机数
z = random_array_1d = np.random.rand(100)  # 生成100个0到1之间的随机数
# 创建 3D 图表
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='lines')])

# 保存为 HTML 文件
pio.write_html(fig, file='3d_plot.html')


In [13]:
import torch
import torch.nn as nn
from net.custome_net import LeNet, MLP, ResNet18,ResNet34, ResNet50, ResNet101, ResNet152

model = ResNet152()

# 获取模型类名并打印
model_name = type(model).__name__
print("Model Name:", model_name)


Model Name: ResNet


In [3]:
from dataset.after_betti import after_get_bars

after_get_bars(base_path=r".\angle_layer_out\0\LeNet")

.\angle_layer_out\0\LeNet\0
观察.\angle_layer_out\0\LeNet\0文件夹,里面有6张图片、1份betti_number.pkl数据。可以计算after_betti。
.\angle_layer_out\0\LeNet\1
观察.\angle_layer_out\0\LeNet\1文件夹,里面有6张图片、1份betti_number.pkl数据。可以计算after_betti。
.\angle_layer_out\0\LeNet\2
观察.\angle_layer_out\0\LeNet\2文件夹,里面有6张图片、1份betti_number.pkl数据。可以计算after_betti。
.\angle_layer_out\0\LeNet\3
观察.\angle_layer_out\0\LeNet\3文件夹,里面有6张图片、1份betti_number.pkl数据。可以计算after_betti。
.\angle_layer_out\0\LeNet\4
观察.\angle_layer_out\0\LeNet\4文件夹,里面有6张图片、1份betti_number.pkl数据。可以计算after_betti。
.\angle_layer_out\0\LeNet\5
观察.\angle_layer_out\0\LeNet\5文件夹,里面有6张图片、1份betti_number.pkl数据。可以计算after_betti。
.\angle_layer_out\0\LeNet\compare_different_bitte_norm_in_same_augmentation.pkl
观察.\angle_layer_out\0\LeNet\compare_different_bitte_norm_in_same_augmentation.pkl文件夹,里面有0张图片、0份betti_number.pkl数据。期望有6张图片，1份betti_number.pkl文件
.\angle_layer_out\0\LeNet\intraspecific_differences
观察.\angle_layer_out\0\LeNet\intraspecific_differences文件夹,里面有0张图片、0份betti_number.pkl

In [10]:
import os
import pickle

def find_and_print_files(file_path: str, target_folder: str):
    if not os.path.isdir(file_path):
        print("Invalid directory path.")
        return

    for root, dirs, files in os.walk(file_path):
        # print(dirs)
        if target_folder in dirs:
            # print(target_folder)
            target_folder_path = os.path.join(root, target_folder)
            print(root.split("\\")[-1])
            pkl_file_path = os.path.join(target_folder_path, "compare_different_bitte_norm_in_same_augmentation.pkl")

            if os.path.exists(pkl_file_path):
                with open(pkl_file_path, 'rb') as pkl_file:
                    try:
                        obj = pickle.load(pkl_file)
                        print(f"File: {pkl_file_path}")
                        print(obj)  # Change this to whatever processing/printing you want to do with the loaded object
                    except Exception as e:
                        print(f"Error loading {pkl_file_path}: {e}")

find_and_print_files(r".\angle_layer_out", "LeNet")

0
File: .\angle_layer_out\0\LeNet\compare_different_bitte_norm_in_same_augmentation.pkl
OrderedDict([('L1-B0', {'bar_number': {'LeNet\\0': 960, 'LeNet\\1': 960, 'LeNet\\2': 960, 'LeNet\\3': 960, 'LeNet\\4': 960, 'LeNet\\5': 960}, 'all_bars_survive_time_sum': {'LeNet\\0': 33740.03588294983, 'LeNet\\1': 7994.791555404663, 'LeNet\\2': 2587.5555539131165, 'LeNet\\3': 1470.4395775794983, 'LeNet\\4': 886.8883979320526, 'LeNet\\5': 92.60332228243351}, 'max_betti_number_epsilon': {'LeNet\\0': (20, 960), 'LeNet\\1': (6, 960), 'LeNet\\2': (2, 960), 'LeNet\\3': (1, 960), 'LeNet\\4': (0, 960), 'LeNet\\5': (0, 960)}, 'birth_len': {'LeNet\\0': 0.0, 'LeNet\\1': 0.0, 'LeNet\\2': 0.0, 'LeNet\\3': 0.0, 'LeNet\\4': 0.0, 'LeNet\\5': 0.0}, 'death_len': {'LeNet\\0': 2.0534291100610367, 'LeNet\\1': 0.5398525039844, 'LeNet\\2': 0.13759537383536635, 'LeNet\\3': 0.09893526045411802, 'LeNet\\4': 0.10838313662102954, 'LeNet\\5': 0.024707362138976046}}), ('L1-B1', {'bar_number': {'LeNet\\0': 719, 'LeNet\\1': 1502,

In [90]:
import os
import pickle
import pandas as pd

def collect_data_into_dict(file_path: str, target_folder: str) -> dict:
    data_dict = {}

    if not os.path.isdir(file_path):
        print("Invalid directory path.")
        return data_dict

    for root, dirs, files in os.walk(file_path):
        if target_folder in dirs:
            target_folder_path = os.path.join(root, target_folder)
            pkl_file_path = os.path.join(target_folder_path, "compare_different_bitte_norm_in_same_augmentation.pkl")

            if os.path.exists(pkl_file_path):
                with open(pkl_file_path, 'rb') as pkl_file:
                    try:
                        obj = pickle.load(pkl_file)
                        key = root.split(os.sep)[-1]
                        data_dict[key] = obj
                    except Exception as e:
                        print(f"Error loading {pkl_file_path}: {e}")

    return data_dict

def restructure_dict(data_dict: dict) -> dict:
    restructured_dict = {}
    for key, value in data_dict.items():
        if isinstance(value, dict):
            for inner_key, inner_value in value.items():
                if inner_key not in restructured_dict:
                    restructured_dict[inner_key] = {}
                restructured_dict[inner_key][key] = inner_value
        else:
            restructured_dict[key] = value
    return restructured_dict


def print_data_dict(data_dict: dict):
    for key, value in data_dict.items():
        pprint(f"Key: {key}")
        pprint(f"Value: {value}")
        pprint("\n")

# Example usage:
file_path = r".\angle_layer_out"
target_folder = "LeNet"

data_dict = collect_data_into_dict(file_path, target_folder)
pprint(data_dict)
# restructured_data_dict = restructure_dict(data_dict)
# df = pd.DataFrame(restructured_data_dict)
df = pd.DataFrame(data_dict)
print(df.shape)
# print(df)
print(df.transpose())
df.transpose().to_pickle(f'{file_path}\{target_folder}.pkl')

{'0': OrderedDict([('L1-B0',
                    {'all_bars_survive_time_sum': {'LeNet\\0': 33740.03588294983,
                                                   'LeNet\\1': 7994.791555404663,
                                                   'LeNet\\2': 2587.5555539131165,
                                                   'LeNet\\3': 1470.4395775794983,
                                                   'LeNet\\4': 886.8883979320526,
                                                   'LeNet\\5': 92.60332228243351},
                     'bar_number': {'LeNet\\0': 960,
                                    'LeNet\\1': 960,
                                    'LeNet\\2': 960,
                                    'LeNet\\3': 960,
                                    'LeNet\\4': 960,
                                    'LeNet\\5': 960},
                     'birth_len': {'LeNet\\0': 0.0,
                                   'LeNet\\1': 0.0,
                                   'LeNet\\2': 0.0,
 

In [76]:
file_path2 = r'.\angle_layer_out\LeNet.pkl'  # 替换成你的.pkl文件路径
with open(file_path2, 'rb') as file2:
    data2 = pickle.load(file2)

# 打印读取的内容
pprint(data2["0"])

L1-B0    {'bar_number': {'LeNet\0': 960, 'LeNet\1': 960...
L1-B1    {'bar_number': {'LeNet\0': 719, 'LeNet\1': 150...
L2-B0    {'bar_number': {'LeNet\0': 960, 'LeNet\1': 960...
L2-B1    {'bar_number': {'LeNet\0': 682, 'LeNet\1': 145...
Name: 0, dtype: object


In [94]:
import pandas as pd




                            0
(A, a1)                     1
(A, a2)                     2
(B, b1)  {'b1a': 3, 'b1b': 4}
(B, b2)  {'b2a': 5, 'b2b': 6}


In [5]:
from dataset.check_betti4net import get_layer_output_betti
import torchvision.transforms as transforms
from net.custome_net import LeNet, MLP

image_size = 32
CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
CIFAR_STD = [0.2023, 0.1994, 0.2010]

model_LeNet = LeNet()
angle_path = "./angle_layer_out"
LeNet_path = f"{angle_path}/LeNet/{0}"
data_transform={'train':transforms.Compose([
                    transforms.RandomRotation(degrees=(0, 5)),
                    transforms.ToTensor(),
                    transforms.Normalize(CIFAR_MEAN, CIFAR_STD)
                    ])}
get_layer_output_betti(model=model_LeNet, seed=15, save_root=LeNet_path, name="LeNet", debug_size=1000, transform=data_transform["train"])
after_get_bars(base_path = LeNet_path)

This is LeNet!!!
这里是得到距离矩阵，是否使用cuda=True
这里是得到距离矩阵，是否使用cuda=True
这里是得到距离矩阵，是否使用cuda=True
这里是得到距离矩阵，是否使用cuda=True


KeyboardInterrupt: 

In [53]:
import os
import pickle

def read_pkl_files_in_folders(path):
    pkl_list = []
    for root, dirs, files in os.walk(path):
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            pkl_file_path = os.path.join(folder_path, 'compare_different_bitte_norm_in_same_augmentation.pkl')
            
            if os.path.exists(pkl_file_path):
                try:
                    with open(pkl_file_path, 'rb') as file:
                        data = pickle.load(file)
                        print(f"File: {pkl_file_path}")
                        print(data)  # 这里打印你读取到的数据
                        pkl_list.append(data)
                except Exception as e:
                    print(f"Error reading {pkl_file_path}: {e}")
            else:
                print(f"No 'compare_different_bitte_norm_in_same_augmentation.pkl' found in {folder_path}")
    return pkl_list

# 调用函数并传入文件路径
folder_path_to_search = './angle_layer_out/LeNet/'  # 替换为你的文件路径
pkl_list = read_pkl_files_in_folders(folder_path_to_search)


File: ./angle_layer_out/LeNet/0\compare_different_bitte_norm_in_same_augmentation.pkl
OrderedDict([('L1-B0', {'bar_number': {'LeNet\\0\\0': 960, 'LeNet\\0\\1': 960, 'LeNet\\0\\2': 960, 'LeNet\\0\\3': 960, 'LeNet\\0\\4': 960, 'LeNet\\0\\5': 960}, 'all_bars_survive_time_sum': {'LeNet\\0\\0': 34720.47020530701, 'LeNet\\0\\1': 8012.8681626319885, 'LeNet\\0\\2': 2952.5307517051697, 'LeNet\\0\\3': 1570.8177386522293, 'LeNet\\0\\4': 1292.3181039094925, 'LeNet\\0\\5': 234.25625654309988}, 'max_betti_number_epsilon': {'LeNet\\0\\0': (19, 960), 'LeNet\\0\\1': (6, 960), 'LeNet\\0\\2': (2, 960), 'LeNet\\0\\3': (1, 960), 'LeNet\\0\\4': (1, 960), 'LeNet\\0\\5': (0, 960)}, 'birth_len': {'LeNet\\0\\0': 0.0, 'LeNet\\0\\1': 0.0, 'LeNet\\0\\2': 0.0, 'LeNet\\0\\3': 0.0, 'LeNet\\0\\4': 0.0, 'LeNet\\0\\5': 0.0}, 'death_len': {'LeNet\\0\\0': 2.644419405064933, 'LeNet\\0\\1': 0.560368252725916, 'LeNet\\0\\2': 0.15342603870619406, 'LeNet\\0\\3': 0.0930437352491537, 'LeNet\\0\\4': 0.08119975953520067, 'LeNet\\0

In [54]:
print(pkl_list)

[OrderedDict([('L1-B0', {'bar_number': {'LeNet\\0\\0': 960, 'LeNet\\0\\1': 960, 'LeNet\\0\\2': 960, 'LeNet\\0\\3': 960, 'LeNet\\0\\4': 960, 'LeNet\\0\\5': 960}, 'all_bars_survive_time_sum': {'LeNet\\0\\0': 34720.47020530701, 'LeNet\\0\\1': 8012.8681626319885, 'LeNet\\0\\2': 2952.5307517051697, 'LeNet\\0\\3': 1570.8177386522293, 'LeNet\\0\\4': 1292.3181039094925, 'LeNet\\0\\5': 234.25625654309988}, 'max_betti_number_epsilon': {'LeNet\\0\\0': (19, 960), 'LeNet\\0\\1': (6, 960), 'LeNet\\0\\2': (2, 960), 'LeNet\\0\\3': (1, 960), 'LeNet\\0\\4': (1, 960), 'LeNet\\0\\5': (0, 960)}, 'birth_len': {'LeNet\\0\\0': 0.0, 'LeNet\\0\\1': 0.0, 'LeNet\\0\\2': 0.0, 'LeNet\\0\\3': 0.0, 'LeNet\\0\\4': 0.0, 'LeNet\\0\\5': 0.0}, 'death_len': {'LeNet\\0\\0': 2.644419405064933, 'LeNet\\0\\1': 0.560368252725916, 'LeNet\\0\\2': 0.15342603870619406, 'LeNet\\0\\3': 0.0930437352491537, 'LeNet\\0\\4': 0.08119975953520067, 'LeNet\\0\\5': 0.07230285436078755}}), ('L1-B1', {'bar_number': {'LeNet\\0\\0': 787, 'LeNet\\0

In [55]:
from collections import OrderedDict
from pprint import pprint

def merge_ordered_dicts(list_of_dicts):
    merged_dict = OrderedDict()

    for current_dict in list_of_dicts:
        nested_dict = merged_dict
        for key, value in current_dict.items():
            if key in nested_dict:
                if isinstance(nested_dict[key], dict) and isinstance(value, dict):
                    # 如果当前键已存在且对应的值都是字典，则递归合并
                    nested_dict[key] = merge_ordered_dicts([nested_dict[key], value])
                else:
                    # 否则，将值更新为一个列表，包含之前的值和新的值
                    nested_dict[key] = [nested_dict[key], value]
            else:
                nested_dict[key] = value

    return merged_dict

# 示例用法
list_of_ordered_dicts = [
    OrderedDict([('key1', OrderedDict([('subkey1', 'value1')])),
                 ('key2', 'value2')]),
    OrderedDict([('key1', OrderedDict([('subkey1', 'value3')])),
                 ('key2', 'value4')])
]

result = merge_ordered_dicts(pkl_list)
pprint(result)


OrderedDict([('L1-B0',
              OrderedDict([('bar_number',
                            OrderedDict([('LeNet\\0\\0', 960),
                                         ('LeNet\\0\\1', 960),
                                         ('LeNet\\0\\2', 960),
                                         ('LeNet\\0\\3', 960),
                                         ('LeNet\\0\\4', 960),
                                         ('LeNet\\0\\5', 960),
                                         ('LeNet\\1\\0', 960),
                                         ('LeNet\\1\\1', 960),
                                         ('LeNet\\1\\2', 960),
                                         ('LeNet\\1\\3', 960),
                                         ('LeNet\\1\\4', 960),
                                         ('LeNet\\1\\5', 960),
                                         ('LeNet\\10\\0', 960),
                                         ('LeNet\\10\\1', 960),
                                         ('LeNet\\1

In [56]:
from collections import OrderedDict

def split_ordered_dict(input_dict):
    # 获取前两层的keys
    first_level_keys = list(input_dict.keys())
    second_level_keys = list(input_dict[first_level_keys[0]].keys())

    # 创建15个新的空字典
    new_dicts = [{} for _ in range(len(first_level_keys) * len(second_level_keys))]

    # 遍历原字典前两层，将值填充到新字典中
    index = 0
    for first_key in first_level_keys:
        for second_key in second_level_keys:
            new_key = first_key + "_" + second_key
            new_dicts[index][new_key] = input_dict[first_key][second_key]
            index += 1

    return new_dicts

# 示例用法
input_ordered_dict = {
    'A': {'1': {'x': 10, 'y': 20}, '2': {'x': 30, 'y': 40}},
    'B': {'1': {'x': 50, 'y': 60}, '2': {'x': 70, 'y': 80}},
    'C': {'1': {'x': 90, 'y': 100}}
}

result2 = split_ordered_dict(result)
for d in result2:
    print(d)
    print("\n")


{'L1-B0_bar_number': OrderedDict([('LeNet\\0\\0', 960), ('LeNet\\0\\1', 960), ('LeNet\\0\\2', 960), ('LeNet\\0\\3', 960), ('LeNet\\0\\4', 960), ('LeNet\\0\\5', 960), ('LeNet\\1\\0', 960), ('LeNet\\1\\1', 960), ('LeNet\\1\\2', 960), ('LeNet\\1\\3', 960), ('LeNet\\1\\4', 960), ('LeNet\\1\\5', 960), ('LeNet\\10\\0', 960), ('LeNet\\10\\1', 960), ('LeNet\\10\\2', 960), ('LeNet\\10\\3', 960), ('LeNet\\10\\4', 960), ('LeNet\\10\\5', 960), ('LeNet\\11\\0', 960), ('LeNet\\11\\1', 960), ('LeNet\\11\\2', 960), ('LeNet\\11\\3', 960), ('LeNet\\11\\4', 960), ('LeNet\\11\\5', 960), ('LeNet\\12\\0', 960), ('LeNet\\12\\1', 960), ('LeNet\\12\\2', 960), ('LeNet\\12\\3', 960), ('LeNet\\12\\4', 960), ('LeNet\\12\\5', 960), ('LeNet\\13\\0', 960), ('LeNet\\13\\1', 960), ('LeNet\\13\\2', 960), ('LeNet\\13\\3', 960), ('LeNet\\13\\4', 960), ('LeNet\\13\\5', 960), ('LeNet\\14\\0', 960), ('LeNet\\14\\1', 960), ('LeNet\\14\\2', 960), ('LeNet\\14\\3', 960), ('LeNet\\14\\4', 960), ('LeNet\\14\\5', 960), ('LeNet\\15\

In [61]:
def convert_dict_to_3d_array(input_dict):
    result_dict = {}
    for main_key, inner_dict in input_dict.items():
        result = []
        for key, value in inner_dict.items():
            new_key = tuple(map(int, key.split('\\')[1:])) + (value,)
            result.append(new_key)
        result_dict[main_key] = result
    return result_dict

# 示例用法
input_dict = {
    'L1-B0_bar_number': OrderedDict([
        ('LeNet\\0\\0', 960),
        ('LeNet\\0\\1', 960),
        ('LeNet\\1\\0', 960),
    ])
}

# print(input_dict)
result3 = convert_dict_to_3d_array(result2[0])
print(result3)


{'L1-B0_bar_number': [(0, 0, 960), (0, 1, 960), (0, 2, 960), (0, 3, 960), (0, 4, 960), (0, 5, 960), (1, 0, 960), (1, 1, 960), (1, 2, 960), (1, 3, 960), (1, 4, 960), (1, 5, 960), (10, 0, 960), (10, 1, 960), (10, 2, 960), (10, 3, 960), (10, 4, 960), (10, 5, 960), (11, 0, 960), (11, 1, 960), (11, 2, 960), (11, 3, 960), (11, 4, 960), (11, 5, 960), (12, 0, 960), (12, 1, 960), (12, 2, 960), (12, 3, 960), (12, 4, 960), (12, 5, 960), (13, 0, 960), (13, 1, 960), (13, 2, 960), (13, 3, 960), (13, 4, 960), (13, 5, 960), (14, 0, 960), (14, 1, 960), (14, 2, 960), (14, 3, 960), (14, 4, 960), (14, 5, 960), (15, 0, 960), (15, 1, 960), (15, 2, 960), (15, 3, 960), (15, 4, 960), (15, 5, 960), (16, 0, 960), (16, 1, 960), (16, 2, 960), (16, 3, 960), (16, 4, 960), (16, 5, 960), (17, 0, 960), (17, 1, 960), (17, 2, 960), (17, 3, 960), (17, 4, 960), (17, 5, 960), (18, 0, 960), (18, 1, 960), (18, 2, 960), (18, 3, 960), (18, 4, 960), (18, 5, 960), (19, 0, 960), (19, 1, 960), (19, 2, 960), (19, 3, 960), (19, 4, 96

In [65]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import os

def plot_3d_interactive(input_dict, file_path):
    # 检查文件夹是否存在，不存在则创建
    if not os.path.exists(file_path):
        os.makedirs(file_path)

    fig = go.Figure()

    for key, data in input_dict.items():
        x, y, z = zip(*data)
        fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers', name=key))

    fig.update_layout(scene=dict(
                        xaxis=dict(title='增强的强度'),
                        yaxis=dict(title='输出的层数'),
                        zaxis=dict(title='指标的大小')),
                      title=list(input_dict.keys())[0])

    file_name = os.path.join(file_path, list(input_dict.keys())[0] + ".html")
    fig.write_html(file_name)

# 示例用法
input_data = {
    'L1-B0_bar_number': [
        (0, 0, 960),
        (0, 1, 960),
        (0, 2, 960),
        (0, 3, 960)
    ]
}

file_save_path = '/your/save/path/'  # 请替换为你想要保存的路径
plot_3d_interactive(input_data, "./net/")


In [2]:
import os

def get_subfolders(path: str) -> None:
    """
    Retrieves subfolder paths and visualizes data from pkl files in each subfolder.

    Args:
    - path (str): The root path.

    Returns:
    - None
    """
    subfolders = [os.path.join(path, folder) for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]
    
    for subfolder in subfolders:
        visualize_aug_layers(subfolder)
        print(subfolder)

get_subfolders("./angle_layer_out/")

./angle_layer_out/LeNet
./angle_layer_out/MLP
./angle_layer_out/ResNet101
./angle_layer_out/ResNet152
./angle_layer_out/ResNet18
./angle_layer_out/ResNet34
./angle_layer_out/ResNet50


In [2]:
from typing import List, Tuple

def replace_indexes_with_values(index_tuple: Tuple[int, ...], value_list: List[int]) -> Tuple[int, ...]:
    """
    Replaces indexes in a tuple with corresponding values from a list.

    Args:
    - index_tuple (Tuple[int, ...]): Tuple containing indexes.
    - value_list (List[int]): List containing values.

    Returns:
    - Tuple[int, ...]: Tuple with indexes replaced by values.
    """
    replaced_values = tuple(value_list[idx] for idx in index_tuple)
    return replaced_values

index_tuple = (0, 2, 1)
value_list = ["10", "20", "30"]

result = replace_indexes_with_values(index_tuple, value_list)
print(result)  # Output: (10, 30, 20)


('10', '30', '20')
